# Setup

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
from tensorflow.keras import layers
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [2]:
def get_encoder(input_shape):
    """ Returns the image encoding model """

    pretrained_model = Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )
    
    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    return encode_model

In [3]:
def load_encoder(weights_path):
    encoder = get_encoder((128, 128, 3))
    encoder.load_weights(weights_path)
    return encoder

# Load the encoder with saved weights
saved_encoder = load_encoder("./model/encoderv2")

In [4]:
def save_face(img_path, dest_folder):
    img = cv2.imread(img_path)
    detector = MTCNN()
    faces = detector.detect_faces(img)

    #fetching the (x,y)co-ordinate and (width-->w, height-->h) of the image
    try:
        x1,y1,w,h = faces[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2 = abs(x1+w)
        y2 = abs(y1+h)

        #locate the co-ordinates of face in the image
        store_face = img[y1:y2,x1:x2]
        store_face = cv2.resize(store_face, (224, 224)) #The VGGFace model expects a 224x224x3 size face image as input, and it outputs a face embedding vector with a length of 2048.
        cv2.imwrite(os.path.join(dest_folder, os.path.basename(img_path)), store_face)
        return "Face Found"
    except IndexError:
        return "Out of Index"

In [5]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128))  # Assuming your model expects 128x128 images
    image = preprocess_input(image)
    return np.expand_dims(image, axis=0)  # Add batch dimension

def compare_two_images(encoder, image_path1, image_path2, threshold=0.7):
    # Preprocess the images
    image1 = preprocess_image(image_path1)
    image2 = preprocess_image(image_path2)

    # Get the embeddings (encodings) for the images
    embedding1 = encoder.predict(image1)
    embedding2 = encoder.predict(image2)

    # Compute the distance between embeddings
    distance = np.sum(np.square(embedding1 - embedding2))
    print(distance)

    # Compare with the threshold
    is_same = distance <= threshold

    #print(f"The images are {'the same' if is_same else 'different'}.")
    return is_same

In [6]:
def verify(input_img, verification_path):
    same = 0
    length = len(os.listdir(verification_path))
    for face in os.listdir(os.path.join(verification_path)):
        veri_img = os.path.join(verification_path, face)
        compare_result = compare_two_images(saved_encoder, input_img, veri_img)
        if compare_result == True:
            same+=1
        #print(compare_result)
    print(same,"/", length)
    percent = (same/length)*100
    print(percent,"% MATCH")
    if percent > 50.0:
        return True
    else:
        return False

In [7]:
def identify_person(input_img, verification_path):
    
    input_img = "./data/input/input_image.jpg"
    verification_path = './data/verifications'

    count_dict = {}
    for a in os.listdir(verification_path):
        count_dict[a] = 0

    for i in range(50):
        print("ITERATION: ", i)
        for folders in os.listdir(verification_path):
            print(folders)
            list = os.listdir(os.path.join(verification_path, folders, "faces"))
            face = list[i]
            print(face)
            veri_img = os.path.join(verification_path, folders, "faces", face)
            result = compare_two_images(saved_encoder, input_img, veri_img)
            print(result)
            print(" ")
            if result == True:
                count_dict[folders] += 1
            for f, count in count_dict.items():
                if count == 5:
                    flag = f
                    print("FIVE OF ", flag, " ARE MATCHED")
                    break
            else:
                continue
            break
        else:
            continue
        break


    print(flag)
    verified = verify(input_img, os.path.join(verification_path, flag, "faces"))
    if verified is True:
        print("Welcome ", flag)
    else:
        print("Please Try Again")

# Verify Person using Webcam

In [8]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Verification', frame)

    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite('./data/input/input_image.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()

input_img = "./data/input/input_image.jpg"
verification_path = './data/verifications'

face = save_face('./data/input/input_image.jpg', 'data/input')
if face != "Out of Index":
    identify_person(input_img, verification_path)
else:
    print("No Face Found")

1/1 [==============================] - 0s 147ms/step
ITERATION:  0
deepu
929b68ab-935e-11ee-af1f-145afc43f0f6.jpg
1/1 [==============================] - 0s 25ms/step
0.1410733
True
 
om
0.jpg
1/1 [==============================] - 0s 24ms/step
1.9103744
False
 
philip
0.jpg
1/1 [==============================] - 0s 24ms/step
1.8921139
False
 
rajeev
0.jpg
1/1 [==============================] - 0s 24ms/step
0.8673299
False
 
ITERATION:  1
deepu
92eac1c7-935e-11ee-ac4e-145afc43f0f6.jpg
1/1 [==============================] - 0s 23ms/step
0.16472085
True
 
om
1.jpg
1/1 [==============================] - 0s 24ms/step
1.9233088
False
 
philip
1.jpg
1/1 [==============================] - 0s 24ms/step
1.8980347
False
 
rajeev
1.jpg
1/1 [==============================] - 0s 23ms/step
0.9532665
False
 
ITERATION:  2
deepu
92f4182b-935e-11ee-b6a1-145afc43f0f6.jpg
1/1 [==============================] - 0s 24ms/step
0.036931008
True
 
om
10.jpg
1/1 [==============================] - 0s 25ms/step
1.

1/1 [==============================] - 0s 25ms/step
0.19274262
1/1 [==============================] - 0s 24ms/step
0.2283418
1/1 [==============================] - 0s 25ms/step
0.09704901
1/1 [==============================] - 0s 25ms/step
0.09704901
1/1 [==============================] - 0s 25ms/step
0.09704901
1/1 [==============================] - 0s 22ms/step
0.10526319
1/1 [==============================] - 0s 23ms/step
0.20088102
50 / 50
100.0 % MATCH
Welcome  deepu
